Libraries and imports 

In [2]:
import pandas as pd
import numpy as np
import regex as re


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import ipywidgets as wd
from IPython.display import display

In [3]:
#importing the data

original_m_data = pd.read_csv(r"C:\Users\nrhhe\Downloads\ml-25m\ml-25m\movies.csv")
movies = original_m_data.copy()

original_r_data = pd.read_csv(r"C:\Users\nrhhe\Downloads\ml-25m\ml-25m\ratings.csv")
ratings = original_r_data.copy()


Taking a look at our datasets 

In [4]:
movies                                                                                      

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [5]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [6]:
# check for null values
movies.isnull().sum()


movieId    0
title      0
genres     0
dtype: int64

In [7]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [8]:
#removing paranthesis from the title column

movies['title_cleaned'] = movies['title'].astype(str).apply(lambda x: re.sub("[^a-zA-Z0-9 ]", "", x))

In [9]:
movies.head()

,movieId,title,genres,title_cleaned
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995


In [10]:
#min_df=0 remove and check 

#creating a tfidf vectorizer
vect = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))

movies_tfidf = vect.fit_transform(movies['title_cleaned'])

In [11]:
#test
title= "Grumpier"
title2="Your Name"
q= vect.transform([title])
q2= vect.transform([title2])
s= cosine_similarity(movies_tfidf, q).flatten()
s

array([0.       , 0.       , 0.4587179, ..., 0.       , 0.       ,
       0.       ])

In [12]:
s2= cosine_similarity(movies_tfidf, q2).flatten()
i = np.argsort(s2)[-10:]
r= movies['title_cleaned'].iloc[i][::-1]
r

42103                                 Your Name 2016
55023                              Tell Me Your Name
56130                            Burn Your Name 2016
44566                      Call Me by Your Name 2017
49627                                   Name Me 2014
51867                              Without Name 2017
2376     Name of the Rose The Name der Rose Der 1986
56382                             A Womans Name 2018
44393                            In the Name of 2013
16003                          Remember My Name 1978
Name: title_cleaned, dtype: object

In [73]:
#given a movie title, return the most simillar movies 

def recommend(title):
    title =  re.sub("[^a-zA-Z0-9 ]", "", title)
    title_vector = vect.transform([title])
    title_cos_sim=cosine_similarity(title_vector, movies_tfidf).flatten()
    indices=np.argsort(title_cos_sim)[-10:]

    #HERER  
    movie_recommend = movies.iloc[indices][::-1]
    
    return movie_recommend

recommend("your name")

,movieId,title,genres,title_cleaned
42103,163134,Your Name. (2016),Animation|Drama|Fantasy|Romance,Your Name 2016
55023,190937,Tell Me Your Name,Horror|Thriller,Tell Me Your Name
56130,193495,Burn Your Name (2016),(no genres listed),Burn Your Name 2016
44566,168492,Call Me by Your Name (2017),Drama|Romance,Call Me by Your Name 2017
49627,179277,Name Me (2014),Adventure|Drama,Name Me 2014
51867,184067,Without Name (2017),Drama|Horror|Mystery,Without Name 2017
2376,2467,"Name of the Rose, The (Name der Rose, Der) (1986)",Crime|Drama|Mystery|Thriller,Name of the Rose The Name der Rose Der 1986
56382,194046,A Woman's Name (2018),Drama,A Womans Name 2018
44393,168128,In the Name of... (2013),Drama|Thriller,In the Name of 2013
16003,84479,Remember My Name (1978),Drama|Thriller,Remember My Name 1978


In [14]:
#given a movie title, return the most simillar movies 

def recommend2(title):
    title =  re.sub("[^a-zA-Z0-9 ]", "", title)
    title_vector = vect.transform([title])
    title_cos_sim=cosine_similarity(title_vector, movies_tfidf).flatten()
    indices=np.argpartition(title_cos_sim,-10)[-10:]

    movie_recommend = movies['title_cleaned'].iloc[indices][::-1]
    
    return movie_recommend

movie_name=wd.Text(
    value='',
    placeholder='Type something',
    description='Movie:',
    disabled=False
)
m_list=wd.Output()

def on_value_change(change):
    with m_list:
        m_list.clear_output()
        name=change['new']
        if len(name)>3:
            display(recommend(name))

movie_name.observe(on_value_change,names='value')
   
    
daata=display(movie_name,m_list)

Text(value='', description='Movie:', placeholder='Type something')

Output()

Here it returns only similar movies by name only not a recomendation system at all 

Collabarative Filtering
- using user ratings for a movie 

Aim
- if we search a movie
- the system  will get other user who liked the movie we searched(users that rated>4)
- then the the system will get the filtered set of movies that those users also liked(other movies that users rated>4) 
- and save them as a percatage (x% amount of users recommended this movie)
- then the system will get all the users that liked(rated>4) the filtered set of movies get the percentage
- by dividing the 2 percentages we get a score for each movie
- then we sort the movies by score, higher the score the more likely the user will like the movie
- idea is larger the difference between the 2 percentages the more likely the user will like the movie(score will be higher)
- the system will return the movies with the highest score

In [15]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [16]:
movie_id=1#163134#Your Name (2016) movie id

In [17]:
#users who liked the movie and gsve it a rating larger than 4

users_alike= ratings[(ratings["movieId"]==movie_id)& (ratings["rating"]>4)]["userId"].unique()

In [18]:
#amount of users who liked the movie
users_alike.shape

(18835,)

In [19]:
#movies that before mentioned users liked
#set of movies that the users liked and also liked our searched movie 
movies_alike= ratings[(ratings["userId"].isin(users_alike)) & (ratings["rating"]>4)]["movieId"]

movies_alike

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [20]:
movies_alike.shape

(1358326,)

In [21]:
movies_alike=movies_alike.value_counts()

In [22]:
#number of movies that the users liked and also liked(4or more rating) our searched movie
movies_alike.shape

(19282,)

In [23]:
users_alike.shape

(18835,)

In [24]:
movies_alike=movies_alike/len(users_alike)
movies_alike

movieId
1         1.000000
318       0.445607
260       0.403770
356       0.370215
296       0.367295
            ...   
128478    0.000053
125125    0.000053
119701    0.000053
107563    0.000053
7625      0.000053
Name: count, Length: 19282, dtype: float64

1.0000 means 100% of the users recomment it 

In [25]:
#getting the movies that are greater than 10%
movies_alike=movies_alike[movies_alike>.1]
movies_alike

movieId
1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: count, Length: 113, dtype: float64

In [26]:
movies_alike_df = pd.DataFrame({'movieId': movies_alike.index, 'also_liked%': movies_alike.values})
movies_alike_df.head()


,movieId,also_liked%
0,1,1.000000
1,318,0.445607
2,260,0.403770
3,356,0.370215
4,296,0.367295


In [27]:
movies_alike_df = pd.merge(movies_alike_df, movies, on='movieId', how='inner')

In [28]:
movies_alike_df.head()

,movieId,also_liked%,title,genres,title_cleaned
0,1,1.000000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,318,0.445607,"Shawshank Redemption, The (1994)",Crime|Drama,Shawshank Redemption The 1994
2,260,0.403770,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,Star Wars Episode IV A New Hope 1977
3,356,0.370215,Forrest Gump (1994),Comedy|Drama|Romance|War,Forrest Gump 1994
4,296,0.367295,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Pulp Fiction 1994


Here you can see Toy Story(the one we hardcoded) and rest of the top movies that are shown not simillar at all. Granted all movies shown are great but they are not simillar to Toy Story at all 

we need to find a way to get  much more simillar movies

In [54]:
movies_alike_df[movies_alike_df['movieId'] == 318]


,movieId,also_liked%,title,genres,title_cleaned
1,318,0.445607,"Shawshank Redemption, The (1994)",Crime|Drama,Shawshank Redemption The 1994


In [30]:

#this all the users who watched the movies in movies_alike set and gave it a rating more than 4

users= ratings[ratings["movieId"].isin(movies_alike.index)&(ratings["rating"]>4)]

In [31]:
users

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
25000062,162541,5618,4.5,1240953299
25000065,162541,5952,5.0,1240952617
25000078,162541,7153,5.0,1240952613
25000081,162541,7361,4.5,1240953484


let's see what percentage of users recommend each movie

In [32]:
#divide
user_recom=users["movieId"].value_counts()/len(users["userId"].unique())
user_recom

movieId
318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: count, Length: 113, dtype: float64

In [33]:
r_percentages=pd.concat([movies_alike,user_recom],axis=1)
r_percentages.columns=["also_liked%","all_user_recom%"]
r_percentages

,also_liked%,all_user_recom%
movieId,,
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877


Create a score (finding a ratio between percentages)

In [34]:
r_percentages["score"]=r_percentages["also_liked%"]/r_percentages["all_user_recom%"]
r_percentages

,also_liked%,all_user_recom%,score
movieId,,,
1,1.000000,0.124728,8.017414
318,0.445607,0.342220,1.302105
260,0.403770,0.222207,1.817089
356,0.370215,0.235266,1.573604
296,0.367295,0.284674,1.290232
...,...,...,...
953,0.103053,0.045792,2.250441
551,0.101195,0.040918,2.473085
1222,0.100876,0.066877,1.508376


In [35]:
#sort by score
r_percentages=r_percentages.sort_values(by="score",ascending=False)
r_percentages

,also_liked%,all_user_recom%,score
movieId,,,
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693


In [36]:
result = r_percentages["score"]
# add a column with same value 1
result = pd.concat([result, pd.Series([1]*len(result), index=result.index)], axis=1)
result

,score,0
movieId,,
1,8.017414,48780
3114,5.225654,48780
2355,4.405452,48780
78499,4.354038,48780
4886,3.320783,48780
...,...,...
2858,1.292845,48780
296,1.290232,48780
79132,1.269693,48780


In [37]:
pivot = result.pivot_table(index=0, columns="movieId", values='score')
pivot

movieId,1,32,34,47,50,110,111,150,260,293,...,33794,48516,48780,50872,58559,59315,60069,68954,78499,79132
0,,,,,,,,,,,,,,,,,,,,,
48780,8.017414,1.602424,2.49966,1.563719,1.374497,1.491075,1.413997,2.10019,1.817089,1.415493,...,1.900462,1.420255,1.466543,2.91456,1.433261,1.92731,2.236221,2.450924,4.354038,1.269693


In [38]:
#getting the top 10 movies
#and merge with movies to get the movie title

r_top=r_percentages.head(10).merge(movies,left_index=True,right_on="movieId",how="inner")
r_top

,also_liked%,all_user_recom%,score,movieId,title,genres,title_cleaned
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,Bugs Life A 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,Monsters Inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,Incredibles The 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion King The 1994


Now it's much better, we can see that the movies are very simillar to Toy Story(cartoons)

In [39]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [40]:
def recommend(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres","movieId"]]


In [70]:
movies[movies['movieId'] == 318]

,movieId,title,genres,title_cleaned
314,318,"Shawshank Redemption, The (1994)",Crime|Drama,Shawshank Redemption The 1994


In [ ]:
test=recommend(1)
test

In [42]:
print(test.iloc[2])


score                                 4.405452
title                     Bug's Life, A (1998)
genres     Adventure|Animation|Children|Comedy
movieId                                   2355
Name: 2264, dtype: object


In [43]:
metadata=pd.read_csv(r"C:\Users\nrhhe\Downloads\ml-25m\ml-25m\movies_metadata.csv")

C:\Users\nrhhe\AppData\Local\Temp\ipykernel_9844\475934067.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata=pd.read_csv(r"C:\Users\nrhhe\Downloads\ml-25m\ml-25m\movies_metadata.csv")


In [44]:
metadata_desc=metadata[["overview","title","id"]]

In [45]:
metadata_desc.head()

,overview,title,id
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,862
1,When siblings Judy and Peter discover an encha...,Jumanji,8844
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,15602
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,31357
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,11862


In [46]:
rating_list=ratings["movieId"].unique()

In [47]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [51]:
movie_count=ratings.groupby("movieId")['rating'].count()
movie_count=movie_count[movie_count>100]
movie_count

movieId
1         57309
2         24228
3         11804
4          2523
5         11714
          ...  
203519      157
204352      185
204542      190
204698      818
205383      252
Name: rating, Length: 10291, dtype: int64

In [49]:
ratings_updated=ratings[ratings["movieId"].isin(movie_count.index)]

In [50]:
ratings_updated

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434
